# __Entregable__: Evaluación de un modelo y su aplicación para predicción

Este es el entregable 2 preparado para el módulo de Estadística (Bloque 2) para la concentración de IA Avanzada<br>

Entregable: Calibración de modelo de serie de tiempo para realizar predicción

Objetivo: Calibrar un modelo ARIMA-SARIMA para la serie del ÍNDICE GLOBAL DE ACTIVIDAD ECONÓMICA de México (CLICK AQUÍ: IGAEEnlaces a un sitio externo.). Esta serie la puedes descargar del sitio del INEGI. Tienes que realizar el proceso de manejo de datos y calibración del modelo para obtener un pronóstico de la economía mexicana hasta el cierre del sexenio actual (SEP 2024).

1. Realizar pruebas estadísticas de estacionariedad y decide el tipo de transformación de la variable
2. Calibra un modelo ARIMA-SARIMA basado en pruebas y gráficos de autocorrelaciones . Explica claramente el proceso de calibración y la razón de cómo va modificando los parámetros.
3. Interpreta los coeficientes y su significancia estadística del modelo calibrado
4. Realiza una predicción de la variable utilizando el modelo calibrado.

In [2]:
# Data management module
import numpy as np
import pandas as pd

In [3]:
igae = pd.read_excel('igae.xls')
igae.head()

,Indicador,Nombre Indicador,Clave área geográfica,Entidad,Municipio,Periodo,Valor,Excepción,Unidad de medida,Frecuencia,Information Status,Referencias
0,6207061409,IGAE. Índice de volumen físico base 2013=100. ...,700,Estados Unidos Mexicanos,Estados Unidos Mexicanos,2022/07,111.274552,NaN,Índice de volumen físico base 2013=100,Mensual,NaN,/a /f1 /r1
1,6207061409,IGAE. Índice de volumen físico base 2013=100. ...,700,Estados Unidos Mexicanos,Estados Unidos Mexicanos,2022/06,112.797501,NaN,Índice de volumen físico base 2013=100,Mensual,NaN,/a /f1 /r1
2,6207061409,IGAE. Índice de volumen físico base 2013=100. ...,700,Estados Unidos Mexicanos,Estados Unidos Mexicanos,2022/05,114.442424,NaN,Índice de volumen físico base 2013=100,Mensual,NaN,/a /f1 /r1
3,6207061409,IGAE. Índice de volumen físico base 2013=100. ...,700,Estados Unidos Mexicanos,Estados Unidos Mexicanos,2022/04,109.529866,NaN,Índice de volumen físico base 2013=100,Mensual,NaN,/a /f1 /r1
4,6207061409,IGAE. Índice de volumen físico base 2013=100. ...,700,Estados Unidos Mexicanos,Estados Unidos Mexicanos,2022/03,111.940152,NaN,Índice de volumen físico base 2013=100,Mensual,NaN,/a /f1 /r1


In [5]:
igae = igae[['Periodo', 'Valor']]
igae.head()

,Periodo,Valor
0,2022/07,111.274552
1,2022/06,112.797501
2,2022/05,114.442424
3,2022/04,109.529866
4,2022/03,111.940152
